<a href="https://colab.research.google.com/github/Gaurav7004/Machine-Learning-Projects/blob/master/Text_Analysis_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -U -q PyDrive

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


In [0]:
import pandas as pd
import numpy as np

In [0]:
cik_list = pd.read_excel("./cik_list.xlsx")
max_row, max_col = cik_list.shape
max_row


152

In [0]:
cik_list.head()


,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME
0,3662,SUNBEAM CORP/FL/,199803,1998-03-06,10-K405,edgar/data/3662/0000950170-98-000413.txt
1,3662,SUNBEAM CORP/FL/,199805,1998-05-15,10-Q,edgar/data/3662/0000950170-98-001001.txt
2,3662,SUNBEAM CORP/FL/,199808,1998-08-13,NT 10-Q,edgar/data/3662/0000950172-98-000783.txt
3,3662,SUNBEAM CORP/FL/,199811,1998-11-12,10-K/A,edgar/data/3662/0000950170-98-002145.txt
4,3662,SUNBEAM CORP/FL/,199811,1998-11-16,NT 10-Q,edgar/data/3662/0000950172-98-001203.txt


In [0]:
pd.set_option('display.max_colwidth',100) 
cik_list.head()


,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME
0,3662,SUNBEAM CORP/FL/,199803,1998-03-06,10-K405,edgar/data/3662/0000950170-98-000413.txt
1,3662,SUNBEAM CORP/FL/,199805,1998-05-15,10-Q,edgar/data/3662/0000950170-98-001001.txt
2,3662,SUNBEAM CORP/FL/,199808,1998-08-13,NT 10-Q,edgar/data/3662/0000950172-98-000783.txt
3,3662,SUNBEAM CORP/FL/,199811,1998-11-12,10-K/A,edgar/data/3662/0000950170-98-002145.txt
4,3662,SUNBEAM CORP/FL/,199811,1998-11-16,NT 10-Q,edgar/data/3662/0000950172-98-001203.txt


In [0]:
link = cik_list.SECFNAME
addr = 'https://www.sec.gov/Archives/'
link =  addr + link
link.head()

0    https://www.sec.gov/Archives/edgar/data/3662/0000950170-98-000413.txt
1    https://www.sec.gov/Archives/edgar/data/3662/0000950170-98-001001.txt
2    https://www.sec.gov/Archives/edgar/data/3662/0000950172-98-000783.txt
3    https://www.sec.gov/Archives/edgar/data/3662/0000950170-98-002145.txt
4    https://www.sec.gov/Archives/edgar/data/3662/0000950172-98-001203.txt
Name: SECFNAME, dtype: object

In [0]:
import requests
import re, string, unicodedata
import nltk
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer

In [0]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
stopset = set(w.upper() for w in stopwords.words('english'))
stopset

{'A',
 'ABOUT',
 'ABOVE',
 'AFTER',
 'AGAIN',
 'AGAINST',
 'AIN',
 'ALL',
 'AM',
 'AN',
 'AND',
 'ANY',
 'ARE',
 'AREN',
 "AREN'T",
 'AS',
 'AT',
 'BE',
 'BECAUSE',
 'BEEN',
 'BEFORE',
 'BEING',
 'BELOW',
 'BETWEEN',
 'BOTH',
 'BUT',
 'BY',
 'CAN',
 'COULDN',
 "COULDN'T",
 'D',
 'DID',
 'DIDN',
 "DIDN'T",
 'DO',
 'DOES',
 'DOESN',
 "DOESN'T",
 'DOING',
 'DON',
 "DON'T",
 'DOWN',
 'DURING',
 'EACH',
 'FEW',
 'FOR',
 'FROM',
 'FURTHER',
 'HAD',
 'HADN',
 "HADN'T",
 'HAS',
 'HASN',
 "HASN'T",
 'HAVE',
 'HAVEN',
 "HAVEN'T",
 'HAVING',
 'HE',
 'HER',
 'HERE',
 'HERS',
 'HERSELF',
 'HIM',
 'HIMSELF',
 'HIS',
 'HOW',
 'I',
 'IF',
 'IN',
 'INTO',
 'IS',
 'ISN',
 "ISN'T",
 'IT',
 "IT'S",
 'ITS',
 'ITSELF',
 'JUST',
 'LL',
 'M',
 'MA',
 'ME',
 'MIGHTN',
 "MIGHTN'T",
 'MORE',
 'MOST',
 'MUSTN',
 "MUSTN'T",
 'MY',
 'MYSELF',
 'NEEDN',
 "NEEDN'T",
 'NO',
 'NOR',
 'NOT',
 'NOW',
 'O',
 'OF',
 'OFF',
 'ON',
 'ONCE',
 'ONLY',
 'OR',
 'OTHER',
 'OUR',
 'OURS',
 'OURSELVES',
 'OUT',
 'OVER',
 'OWN',
 'R

In [0]:
import glob
path = "StopWords*.txt"
glob.glob(path)
for filename in glob.glob(path):
    with open(filename, 'r') as f:
        text = f.read()
        text = re.sub(r"\s+\|\s+[\w]*" , "", text)        
        stopset.update(text.upper().split())

In [0]:
from nltk.corpus import cmudict
nltk.download('cmudict')
def syllables(word):
    count = 0
    vowels = 'aeiouy'
    word = word.lower()
    if word[0] in vowels:
        count +=1
    for index in range(1,len(word)):
        if word[index] in vowels and word[index-1] not in vowels:
            count +=1
    if word.endswith('e'):
        count -= 1
    if word.endswith('le'):
        count+=1
    if count == 0:
        count +=1
    return count

def nsyl(word):
    try:
        return max([len(list(y for y in x if y[-1].isdigit())) for x in d[word.lower()]])
    except KeyError:
        #if word not found in cmudict
        return syllables(word)    

[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Package cmudict is already up-to-date!


In [0]:
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

In [0]:
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

In [0]:
def remove_digits(text):
    return re.sub('[\d%/$]', '', text)

In [0]:
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    text = remove_digits(text)
    return text

In [0]:
def remove_non_ascii(words):
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

In [0]:
def to_upper_case(words):
    new_words = []
    for word in words:
        new_word = word.upper()
        new_words.append(new_word)
    return new_words

In [0]:
def remove_punctuation(words):
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

In [0]:
def remove_stopwords(words):
    new_words = []
    for word in words:
        if word not in stopset:
            new_words.append(word)
    return new_words

In [0]:
def normalize(words):
    words = remove_non_ascii(words)
    words = to_upper_case(words)
    words = remove_punctuation(words)
    words = remove_stopwords(words)
    return words

In [0]:
mda = "Management's Discussion and Analysis"
qqdmr = "Quantitative and Qualitative Disclosures about Market Risk"
rf = "Risk Factors"
section_name = ['mda','qqdmr',"rf"]
section = [mda.upper(),qqdmr.upper(),rf.upper()]
variables = ['positive_score','negative_score','polarity_score','average_sentence_length', 'percentage_of_complex_words',\
                   'fog_index','complex_word_count','word_count','uncertainty_score','constraining_score', 'positive_word_proportion',\
                   'negative_word_proportion', 'uncertainty_word_proportion', 'constraining_word_proportion' ]

In [0]:
import itertools
constraining_words_whole_report = pd.Series(name='constraining_words_whole_report')

df_col = [sec.lower() + '_' + var for sec,var in itertools.product(section_name,variables) ]
df = pd.DataFrame(columns=df_col)

In [0]:

master_dict = pd.read_csv('./LoughranMcDonald_MasterDictionary.csv', index_col= 0)

In [0]:
constraining_dict = set(pd.read_excel('./constraining_dictionary.xlsx',index_col = 0).index)
uncertainty_dict = set(pd.read_excel('./uncertainty_dictionary.xlsx', index_col = 0).index)

In [0]:


cik_list.loc[64]



CIK                                             4962
CONAME                           AMERICAN EXPRESS CO
FYRMO                                         201407
FDATE                            2014-07-30 00:00:00
FORM                                            10-Q
SECFNAME    edgar/data/4962/0001193125-14-286961.txt
Name: 64, dtype: object

In [0]:
for i in range(max_row):
     text = requests.get(link[i]).text
     text
     file_name = 'form' + str(i)
     f = open(file_name, 'a+')
     f.write(text)
     f.close()

In [0]:
for i in range(max_row):
    #print(i)
    file_name = './form' + str(i)
    text = open(file_name,'r').read()
    print('reading..',end = "done ")

reading..done reading..done reading..done reading..done reading..done reading..done reading..done reading..done reading..done reading..done reading..done reading..done reading..done reading..done reading..done reading..done reading..done reading..done reading..done reading..done reading..done reading..done reading..done reading..done reading..done reading..done reading..done reading..done reading..done reading..done reading..done reading..done reading..done reading..done reading..done reading..done reading..done reading..done reading..done reading..done reading..done reading..done reading..done reading..done reading..done reading..done reading..done reading..done reading..done reading..done reading..done reading..done reading..done reading..done reading..done reading..done reading..done reading..done reading..done reading..done reading..done reading..done reading..done reading..done reading..done reading..done reading..done reading..done reading..done reading..done reading..done readin

In [0]:

df.loc[i] = np.zeros(42)
    
for j in range(3):
    if i in [63,64]:
        continue
    print(i,j,sep= '|',end = " ")
    exp = r".*(?P<start>ITEM [\d]\. " + re.escape(section[j]) + r")(?P<MDA>.*)(?P<body>[\s\S]*)(?P<end>ITEM \d|SIGNATURES)"
    regexp = re.compile(exp)
    s = regexp.search(text)
        
    if s:
        data = s.group('body')
        text = denoise_text(data)
        sent_list = sent_tokenize(text)
        sentence_length = len(sent_list)

        sample = text.split()
        sample = normalize(sample)
        word_count = len(sample)
        complex_word_count = 0
            
        for word in sample:
            if nsyl(word.lower()) > 2:
                complex_word_count += 1
            
        average_sentence_length = word_count/sentence_length
        percentage_of_complex_words = complex_word_count/word_count
        fog_index = 0.4 * (average_sentence_length + percentage_of_complex_words)
            
        positive_score = 0
        negative_score = 0
        uncertainty_score = 0
        constraining_score = 0
        for word in sample:
            if word in master_dict.index:
                    
                if master_dict.loc[word].Positive > 0:
                    positive_score += 1
                if master_dict.loc[word].Negative > 0:
                    negative_score += 1
                if word in uncertainty_dict:
                    uncertainty_score += 1
                if word in constraining_dict:
                    constraining_score += 1
            
        polarity_score = (positive_score-negative_score)/(positive_score + negative_score + .000001)
        positive_word_proportion = positive_score/word_count
        negative_word_proportion = negative_score/word_count
        uncertainty_word_proportion = uncertainty_score/word_count
        constraining_word_proportion = constraining_score/word_count
            
        df.loc[i][section_name[j].lower() + "_positive_score"] = positive_score
        df.loc[i][section_name[j].lower() + "_negative_score"] = negative_score
        df.loc[i][section_name[j].lower() + "_polarity_score"] = polarity_score
        df.loc[i][section_name[j].lower() + "_average_sentence_length"] = average_sentence_length
        df.loc[i][section_name[j].lower() + "_percentage_of_complex_words"] = percentage_of_complex_words
        df.loc[i][section_name[j].lower() + "_fog_index"] = fog_index
        df.loc[i][section_name[j].lower() + "_complex_word_count"] = complex_word_count
        df.loc[i][section_name[j].lower() + "_word_count"] = word_count
        df.loc[i][section_name[j].lower() + "_uncertainty_score"] = uncertainty_score
        df.loc[i][section_name[j].lower() + "_constraining_score"] = constraining_score
        df.loc[i][section_name[j].lower() + "_positive_word_proportion"] = positive_word_proportion
        df.loc[i][section_name[j].lower() + "_negative_word_proportion"] = negative_word_proportion
        df.loc[i][section_name[j].lower() + "_uncertainty_word_proportion"] = uncertainty_word_proportion
        df.loc[i][section_name[j].lower() + "_constraining_word_proportion"] = constraining_word_proportion 

151|0 151|1 151|2 

In [0]:
for i in range(max_row):
    print(i,end = " ")
    file_name = './form' + str(i)
    text = open(file_name,'r').read()
    print('reading..',end = " ")
    
    constraining_words_whole_report.loc[i] = 0
    constraining_words_whole_report_count = 0
    for word in denoise_text(text).split():
        if word in constraining_dict:
            constraining_words_whole_report_count += 1
    print('here...',end = "  ")
    constraining_words_whole_report.loc[i] = constraining_words_whole_report_count

0 reading.. here...  1 reading.. here...  2 reading.. here...  3 reading.. here...  4 reading.. here...  5 reading.. here...  6 reading.. here...  7 reading.. here...  8 reading.. here...  9 reading.. here...  10 reading.. here...  11 reading.. here...  12 reading.. here...  13 reading.. here...  14 reading.. here...  15 reading.. here...  16 reading.. here...  17 reading.. here...  18 reading.. here...  19 reading.. here...  20 reading.. here...  21 reading.. here...  22 reading.. here...  23 reading.. here...  24 reading.. here...  25 reading.. here...  26 reading.. here...  27 reading.. here...  28 reading.. here...  29 reading.. here...  30 reading.. here...  31 reading.. here...  32 reading.. here...  33 reading.. here...  34 reading.. here...  35 reading.. here...  36 reading.. here...  37 reading.. here...  38 reading.. here...  39 reading.. here...  40 reading.. here...  41 reading.. here...  42 reading.. here...  43 reading.. here...  44 reading.. here...  45 reading.. here...

In [0]:
df = pd.concat([cik_list,df,constraining_words_whole_report], axis = 1)
df.shape

(152, 49)

In [0]:
df.head(10)

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME,mda_positive_score,mda_negative_score,mda_polarity_score,mda_average_sentence_length,mda_percentage_of_complex_words,mda_fog_index,mda_complex_word_count,mda_word_count,mda_uncertainty_score,mda_constraining_score,mda_positive_word_proportion,mda_negative_word_proportion,mda_uncertainty_word_proportion,mda_constraining_word_proportion,qqdmr_positive_score,qqdmr_negative_score,qqdmr_polarity_score,qqdmr_average_sentence_length,qqdmr_percentage_of_complex_words,qqdmr_fog_index,qqdmr_complex_word_count,qqdmr_word_count,qqdmr_uncertainty_score,qqdmr_constraining_score,qqdmr_positive_word_proportion,qqdmr_negative_word_proportion,qqdmr_uncertainty_word_proportion,qqdmr_constraining_word_proportion,rf_positive_score,rf_negative_score,rf_polarity_score,rf_average_sentence_length,rf_percentage_of_complex_words,rf_fog_index,rf_complex_word_count,rf_word_count,rf_uncertainty_score,rf_constraining_score,rf_positive_word_proportion,rf_negative_word_proportion,rf_uncertainty_word_proportion,rf_constraining_word_proportion,constraining_words_whole_report
0,3662,SUNBEAM CORP/FL/,199803,1998-03-06,10-K405,edgar/data/3662/0000950170-98-000413.txt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55
1,3662,SUNBEAM CORP/FL/,199805,1998-05-15,10-Q,edgar/data/3662/0000950170-98-001001.txt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41
2,3662,SUNBEAM CORP/FL/,199808,1998-08-13,NT 10-Q,edgar/data/3662/0000950172-98-000783.txt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,3662,SUNBEAM CORP/FL/,199811,1998-11-12,10-K/A,edgar/data/3662/0000950170-98-002145.txt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38
4,3662,SUNBEAM CORP/FL/,199811,1998-11-16,NT 10-Q,edgar/data/3662/0000950172-98-001203.txt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
5,3662,SUNBEAM CORP/FL/,199811,1998-11-25,10-Q/A,edgar/data/3662/0000950170-98-002278.txt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
6,3662,SUNBEAM CORP/FL/,199812,1998-12-22,10-Q,edgar/data/3662/0000950170-98-002401.txt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8
7,3662,SUNBEAM CORP/FL/,199812,1998-12-22,10-Q,edgar/data/3662/0000950170-98-002402.txt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
8,3662,SUNBEAM CORP/FL/,199903,1999-03-31,NT 10-K,edgar/data/3662/0000950172-99-000362.txt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
9,3662,SUNBEAM CORP/FL/,199905,1999-05-11,10-K,edgar/data/3662/0000950170-99-000775.txt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15


In [0]:
ls